In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
np.random.seed(0)

In [20]:
class Person():
    def __init__ (self, random_pct = np.random.random()):
        self.is_alive = True
        self.is_vaccinated = False
        self.is_infected = False
        self.has_been_infected = False
        self.newly_infected = False
        self.random_pct = random_pct
    
    def get_vaccinated(self, pct_vaccinated):
        print(1 - pct_vaccinated)
        if self.random_pct >= (1 - pct_vaccinated):
            self.is_vaccinated = True
        else:
            self.is_vaccinated = False
            

In [21]:
joe_shmoe = Person()

In [31]:
joe_shmoe.get_vaccinated(.75)

0.25


In [32]:
joe_shmoe.is_vaccinated

True

In [36]:
class Simulation():
    def __init__(self,
                 population_size,
                 disease_name,
                 r0,
                 mortality_rate,
                 total_time_steps,
                 pct_vaccinated,
                 num_initial_infected):
        self.r0 = r0/100
        self.disease_name = disease_name
        self.mortality_rate = mortality_rate
        self.total_time_steps = total_time_steps
        self.current_time_step = 0
        self.total_infected_counter = 0
        self.current_infected_counter = 0
        self.dead_counter = 0
        self.population = []
        self.time_step_statistics_df = pd.DataFrame()

        for i in range(population_size):
            # Create new person
            new_person = Person()
            # We'll add infected persons to our simulation first. Check if the
            # current number of infected are equal to the num_initial_infected
            # parameter. If not, set new_person to be infected
            if self.current_infected_counter <= num_initial_infected:
                new_person.is_infected = True
                # don't forget to increment both infected counters!
                self.total_infected_counter += 1
                self.current_infected_counter += 1
            # if new_person is not infected, determine if they are vaccinated 
            # or not by using their `get_vaccinated` method.
            # Then, append new_person to self.population
            else:
                new_person.get_vaccinated(self.pct_vaccinated)
            self.population.append(new_person)
       
        print("-" * 50)
        print("Simulation Initiated!")
        print("-" * 50)
        self._get_sim_statistics()
    
    def _get_sim_statistics(self):
        num_infected = 0
        num_dead = 0
        num_vaccinated = 0
        num_immune = 0
        for i in self.population:
            if i.is_infected:
                num_infected += 1
            if not i.is_alive:
                num_dead += 1
            if i.is_vaccinated:
                num_vaccinated += 1
                num_immune += 1
            if i.has_been_infected:
                num_immune += 1
        assert num_infected == self.current_infected_counter
        assert num_dead == self.dead_counter

        print("")
        print("Summary Statistics for Time Step {}".format(self.current_time_step))
        print("")
        print("-" * 50)
        print("Disease Name: {}".format(self.disease_name))
        print("R0: {}".format(self.r0 * 100))
        print("Mortality Rate: {}%".format(self.mortality_rate * 100))
        print("Total Population Size: {}".format(len(self.population)))
        print("Total Number of Vaccinated People: {}".format(num_vaccinated))
        print("Total Number of Immune: {}".format(num_immune))
        print("Current Infected: {}".format(num_infected))
        print("Deaths So Far: {}".format(num_dead))   

In [38]:
def infected_interaction(self, infected_person):
    num_interactions = 0
    while num_interactions < 100:
        # Randomly select a person from self.population
        random_person = np.random.choice(self.population)
        # This only counts as an interaction if the random person selected is 
        # alive.  If the person is dead, do nothing, and the counter doesn't 
        # increment, repeating the loop and selecting a new person at random.
        # check if the person is alive.
        if random_person.is_alive == True:
            # CASE: Random person is not vaccinated, and has not been infected
            # before, making them vulnerable to infection
            if is_vaccinated == False and has_been_infected == False:
                # Generate a random number between 0 and 1
                random_number = np.random.random()
                # If random_number is greater than or equal to (1 - self.r0), 
                # set random person as newly_infected
                if random_number >= (1 - self.r0):
                    random_person.newly_infected = True
            # Don't forget to increment num_interactions, and make sure it's 
            # at this level of indentation
            num_interactions += 1
        else:
            continue

# Adds this function to our Simulation class
Simulation.infected_interaction = infected_interaction

In [41]:
def _resolve_states(self):
    # Iterate through each person in the population
    for person in population:
        # We only need to worry about the people that are still alive
        if is_alive == True: 
            # CASE: Person was infected this round. We need to stochastically 
            # determine if they die or recover from the disease
            # Check if person is_infected
            if is_infected == True:
                # Generate a random number BETWEEN 0 AND 1
                random_number = np.random.random()
                # If random_number is >= (1 - self.mortality_rate), set the 
                # person to dead and increment the simulation's death counter
                if random_number >= (1 - self.mortality_rate):
                    # Set is_alive and is_infected both to False
                    person.is_alive = False
                    person.is_infected = False
                    # Don't forget to increment self.dead_counter, and 
                    # decrement self.current_infected_counter
                    self.dead_counter += 1
                    self.current_infected_counter -= 1
                else:
                    # CASE: They survive the disease and recover. Set 
                    # is_infected to False and has_been_infected to True
                    person.is_infected = False
                    person.has_been_infected = True
                    # Don't forget to decrement self.current_infected_counter!
                    self.current_infected_counter -= 1
            # CASE: Person was newly infected during this round, and needs to 
            # be set to infected before the start of next round
            elif newly_infected == True:
                # Set is_infected to True, newly_infected to False, and 
                # increment both self.current_infected_counter and 
                # self.total_infected_counter
                person.is_infected = True
                person.newly_infected = False
                self.current_infected_counter += 1
                self.total_infected_counter += 1
                
Simulation._resolve_states = _resolve_states